## Connect to Meilisearch

In [30]:
import meilisearch

meilisearch_client = meilisearch.Client("http://localhost:7700", "aStrongMasterKey")
print("Successfully connected to Meilisearch!")

Successfully connected to Meilisearch!


## Get all documents

In [31]:
index = meilisearch_client.index("articles")

all_documents = []
limit = 1000
offset = 0

print(f"Fetching all documents from the '{index.uid}' index...")

while True:
    response = index.get_documents({"limit": limit, "offset": offset})
    batch = response.results
    all_documents.extend(batch)

    if len(batch) < limit:
        break

    offset += limit

print(f"Total documents retrieved: {len(all_documents)}")

Fetching all documents from the 'articles' index...
Total documents retrieved: 10


## Remove a field

In [32]:
documents_to_update = []

field_to_remove = "chunk_number"
for document in all_documents:
    document_dict = document.__dict__
    if field_to_remove in document_dict:
        del document_dict[field_to_remove]
    documents_to_update.append(document_dict)

Delete all documents from Meilisearch and replace them with the updated documents.

In [33]:
task = index.delete_all_documents()
meilisearch_client.wait_for_task(task.task_uid)

task = index.add_documents(documents_to_update)
meilisearch_client.wait_for_task(task.task_uid)

Task(uid=1021, index_uid='articles', status='succeeded', type='documentAdditionOrUpdate', details={'receivedDocuments': 10, 'indexedDocuments': 10}, error=None, canceled_by=None, duration='PT0.411521397S', enqueued_at=datetime.datetime(2025, 11, 2, 20, 45, 32, 916426), started_at=datetime.datetime(2025, 11, 2, 20, 45, 32, 918670), finished_at=datetime.datetime(2025, 11, 2, 20, 45, 33, 330192))

Let's verify that the field is no longer present in the documents.

In [34]:
offset = 0
new_all_documents = []

print(f"Fetching all documents from the '{index.uid}' index...")

while True:
    response = index.get_documents({"limit": limit, "offset": offset})
    batch = response.results
    new_all_documents.extend(batch)

    if len(batch) < limit:
        break

    offset += limit

print(f"Total documents retrieved: {len(new_all_documents)}")

Fetching all documents from the 'articles' index...
Total documents retrieved: 10


In [35]:
for document in new_all_documents:
    assert field_to_remove not in document.__dict__

No errors, that means the field was removed successfully!